In [1]:
import pandas as pd  
import numpy as np 
import torch 
import matplotlib.pyplot as plt 

In [2]:
TrainCsv =  pd.read_csv("/home/jik19004/FilesToRun/PercentileDataAnalysis/newTrainDf.csv")[["Hour","WeekDay or Weekend","Sknt_hourly","Tmpf_Hourly", "100", "Demand"]]
ValidationCsv = pd.read_csv("/home/jik19004/FilesToRun/PercentileDataAnalysis/newValidationDf.csv")[["Hour","WeekDay or Weekend","Sknt_hourly","Tmpf_Hourly", "100","Demand"]]
TestingCsv = pd.read_csv("/home/jik19004/FilesToRun/PercentileDataAnalysis/newTestingDf.csv")[["Hour","WeekDay or Weekend","Sknt_hourly","Tmpf_Hourly", "100","Demand"]]

In [3]:
TrainUncertainty = pd.read_csv("/home/jik19004/FilesToRun/StackedEnsemblePred/StackedTrainVer2.csv")[["energy_demand_mean", "lower_bound","upper_bound"]]
ValUncertainty = pd.read_csv("/home/jik19004/FilesToRun/StackedEnsemblePred/StackedValVer2.csv")[["energy_demand_mean", "lower_bound","upper_bound"]]
TestUncertainty = pd.read_csv("/home/jik19004/FilesToRun/StackedEnsemblePred/StackedTestVer2.csv")[["energy_demand_mean", "lower_bound","upper_bound"]]

TrainCsv = pd.concat([TrainCsv, TrainUncertainty], axis=1)
ValidationCsv = pd.concat([ValidationCsv, ValUncertainty], axis=1)
TestingCsv = pd.concat([TestingCsv, TestUncertainty], axis=1)

In [4]:
display(TrainCsv)

,Hour,WeekDay or Weekend,Sknt_hourly,Tmpf_Hourly,100,Demand,energy_demand_mean,lower_bound,upper_bound
0,18,0,4.875000,44.285000,3640.257568,3858.000,3501.8867,3347.3906,3656.3828
1,19,0,4.958333,42.912500,3609.803223,3757.000,3435.8843,3279.0970,3592.6716
2,20,0,4.791667,42.732500,3521.481201,3634.000,3347.0496,3205.0925,3489.0066
3,21,0,5.250000,42.350000,3349.622559,3471.000,3203.9760,3077.3188,3330.6333
4,22,0,4.875000,42.507500,3082.854248,3238.000,2945.3070,2831.8362,3058.7776
...,...,...,...,...,...,...,...,...,...
61340,19,0,5.625000,6.875000,4387.694336,4682.333,4193.2417,4020.4412,4366.0420
61341,20,0,5.125000,5.750000,4279.734375,4516.193,4142.3975,3976.5447,4308.2505
61342,21,0,5.625000,5.500000,4215.001465,4349.559,4048.0935,3877.6084,4218.5786
61343,22,0,4.250000,5.137500,4041.653076,4173.255,3877.3296,3710.7612,4043.8980


In [5]:
display(pd.read_csv("/home/jik19004/FilesToRun/StackedEnsemblePred/StackedTrainVer2.csv"))

,Hour,WeekDay or Weekend,Sknt_hourly,Tmpf_Hourly,Demand,energy_demand_mean,lower_bound,upper_bound
0,18,0,4.875000,44.285000,3858.000,3501.8867,3347.3906,3656.3828
1,19,0,4.958333,42.912500,3757.000,3435.8843,3279.0970,3592.6716
2,20,0,4.791667,42.732500,3634.000,3347.0496,3205.0925,3489.0066
3,21,0,5.250000,42.350000,3471.000,3203.9760,3077.3188,3330.6333
4,22,0,4.875000,42.507500,3238.000,2945.3070,2831.8362,3058.7776
...,...,...,...,...,...,...,...,...
61340,19,0,5.625000,6.875000,4682.333,4193.2417,4020.4412,4366.0420
61341,20,0,5.125000,5.750000,4516.193,4142.3975,3976.5447,4308.2505
61342,21,0,5.625000,5.500000,4349.559,4048.0935,3877.6084,4218.5786
61343,22,0,4.250000,5.137500,4173.255,3877.3296,3710.7612,4043.8980


In [6]:
from sklearn.preprocessing import StandardScaler
TrainY = TrainCsv["Demand"]
ValidationY = ValidationCsv["Demand"]
TestingY = TestingCsv["Demand"]

TrainX = TrainCsv.drop("Demand", axis = 1)
ValidationX = ValidationCsv.drop("Demand", axis = 1)
TestingX = TestingCsv.drop("Demand", axis = 1)

In [7]:
def return_sequences(data, outputData, input_n_steps, output_n_steps): 
    X = [] 
    Y = [] 
    length = len(data) 
    for i in range(0,length, 1): 
        input_indx = i + input_n_steps 
        output_indx = input_indx + output_n_steps  
        if (output_indx > len(data)): # we need to have equally split sequences.  
            break               # The remaining data that cannot fit into a fixed 
                                # sequence will immediately be cut! 
        else:
            Xsample = data.iloc[i:input_indx, :] # get the previous data 
            Ysample = outputData[input_indx:output_indx] 
            X.append(Xsample) 
            Y.append(Ysample) 
    X = np.asarray(X).astype('float64') 
    Y = np.asarray(Y).astype('float64') 
    return (X, Y) 

In [8]:
def scaleTheData(data):
    scaler = StandardScaler()
    # split the data first. 
    data2 = scaler.fit_transform(data)
    data = pd.DataFrame(data2, columns = data.columns)
    return data

def splitDataAndScale(TrainData, ValidationData, TestData, TrainY, ValidationY, TestY,):    
    # Janruary 1st 2018. 

    TrainData = scaleTheData(TrainData.copy())
    ValidationData = scaleTheData(ValidationData.copy())
    TestingData = scaleTheData(TestData.copy())

    TrainingSequences = return_sequences(TrainData, TrainY, 18, 1) 

    TransformedTrainingData = TrainingSequences[0]
    TransformedTrainingOutput = TrainingSequences[1]

    ValidationSequences = return_sequences(ValidationData, ValidationY, 18, 1)

    TransformedValidationData = ValidationSequences[0]
    TransformedValidationOutput = ValidationSequences[1]

    TestingSequences = return_sequences(TestingData, TestY, 18, 1)

    TransformedTestingData = TestingSequences[0]
    TransformedTestingOutput = TestingSequences[1]


    return (TransformedTrainingData, TransformedTrainingOutput, TransformedValidationData, TransformedValidationOutput, 
    TransformedTestingData, TransformedTestingOutput)

In [9]:
data = splitDataAndScale(TrainX, ValidationX, TestingX, TrainY, ValidationY, TestingY) # returns a tuple. 
TrainX = data[0]
TrainY = data[1]

ValX = data[2]
ValY = data[3]

TestX = data[4]
TestY = data[5]


In [10]:
import torch
class LSTMModel(torch.nn.Module):
    def __init__(self, num_layers, LSTMNeurons, params, output_num =1):
        super(LSTMModel, self).__init__()
        self.LSTM1 = torch.nn.LSTM(input_size = 8, hidden_size = LSTMNeurons, num_layers = 1, bias = True, batch_first = True)
        self.LSTM2 = torch.nn.LSTM(input_size = LSTMNeurons, hidden_size = LSTMNeurons, num_layers = 1, bias = True, batch_first = True, bidirectional = False)
        self.batchNorm0 = torch.nn.BatchNorm1d(num_features = 18)


        input_size = LSTMNeurons
        layers = []
        for i in range(num_layers):
            num_units = params[i]
            layers.append(torch.nn.Linear(input_size, num_units, bias = True))
            layers.append(torch.nn.BatchNorm1d(num_features = 18))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(0.3))

            input_size = num_units
        self.intermediateLayers = torch.nn.Sequential(*layers)
        self.Linear1 = torch.nn.Linear(in_features = input_size, out_features = LSTMNeurons, bias = True)
        self.Activation1 = torch.nn.ReLU()
        self.Dropout = torch.nn.Dropout(0.3)


        self.Linear2 = torch.nn.Linear(in_features = LSTMNeurons * 18, out_features = output_num, bias = True)
        self.Activation2 = torch.nn.ReLU()



    def forward(self, val):

        x = self.LSTM1(val)
        x = self.LSTM2(x[0], (x[1][0],x[1][1])) #(x[1][0], x[1][1])
        x = self.batchNorm0(x[0])
        x = self.Dropout(x)

        y = x.clone() # skip connection.

        x = self.intermediateLayers(x)
        x = self.Linear1(x)
        x = self.Activation1(x)
        x = self.Dropout(x)
        x = x.view(-1, x.size(1) * x.size(2))
        y = y.view(-1, y.size(1) * y.size(2))
        x = self.Linear2(torch.add(x,y))
        x = self.Activation2(x)

        return x

In [11]:
from torch.utils.data import Dataset, DataLoader
import torch 
class TimeSeriesDataset(Dataset):
    def __init__(self, data, output):
        data = torch.tensor(data).float(); 
        output = torch.tensor(output).float() 
        self.data = data 
        self.output = output; 

    def __len__(self):
        return len(self.data) 

    def __getitem__(self, idx):
        x = self.data[idx];  
        y = self.output[idx]; 
        
        return x, y; 

TrainDataset = TimeSeriesDataset(np.array(TrainX), np.array(TrainY))
ValidationDataset = TimeSeriesDataset(np.array(ValX), np.array(ValY))
TestingDataset = TimeSeriesDataset(np.array(TestX), np.array(TestY))

TrainingLoader = DataLoader(TrainDataset, batch_size=256, shuffle=True)
ValidationLoader = DataLoader(ValidationDataset, batch_size=256, shuffle=True)
TestingLoader = DataLoader(TestingDataset, batch_size=256, shuffle=True)

In [12]:
def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item() * target.size(0)
    return running_loss / len(val_loader.dataset)

def Train_and_Evaluate(train_loader, val_loader, device, params1, params2, numEpochs, early_stop_epochs):
    #num_layers, dropout = 0.1, outfeatures1 = 16, outfeatures2 = 16, outfeatures3 = 16, outfeatures4 = 16, dim_feedforward = 2048, output_num = 1
    model = LSTMModel(num_layers = params1[0], LSTMNeurons = params1[1], params = params2)
    model = model.to(device);
    LossFunction = torch.nn.L1Loss();
    best_val_loss = float('inf')
    early_stop_count = 0


    Optimizer = torch.optim.Adam(params = model.parameters(), weight_decay = 0.01)
    for epoch in range(0,numEpochs):
        model.train()
        Training_Loss = 0;
        total_samples = 0;
        for input, output in train_loader:
            input = input.to(device);
            output = torch.squeeze(output, 1);
            output = output.to(device);
            predictedVal = model(input)
            predictedVal = torch.squeeze(predictedVal, 1)
            Optimizer.zero_grad();
            batchLoss = LossFunction(predictedVal, output);
            batchLoss.backward();
            Optimizer.step();
            Training_Loss += batchLoss * output.size(0) #* output.size(0);
            total_samples += output.size(0)
        Training_Loss = Training_Loss.item()/total_samples


        Validation_Loss = 0;
        print("passed ", epoch, "epoch", "Training Loss: ", Training_Loss," ", end = "")
        with torch.no_grad():
            model.eval()
            total_val_samples = 0;
            Validation_Loss = 0;
            for val_input, val_output in val_loader:
                val_input = val_input.to(device);
                val_output = torch.squeeze(val_output,1);
                val_output = val_output.to(device);
                predictedVal = model(val_input)
                predictedVal = torch.squeeze(predictedVal, 1)
                Validation_Loss += LossFunction(val_output, predictedVal) * val_output.size(0)
                total_val_samples += val_output.size(0)
            Validation_Loss = Validation_Loss/total_val_samples
            print("Validation Loss: ", Validation_Loss)

            if Validation_Loss < best_val_loss:
                best_val_loss = Validation_Loss
                torch.save(model, "/home/jik19004/FilesToRun/StackedEnsemblePred/StackedLSTMver2")
                early_stop_count = 0;
            else:
                early_stop_count +=1
            if early_stop_count >= early_stop_epochs:
                return best_val_loss;
    return best_val_loss;

def predict(model, data_loader, device):
    model.eval()
    predictions = []
    act_outputs = []
    with torch.no_grad():
        for data, _ in data_loader:
            data = data.to(device)
            output = model(data)
            predictions.append(output.cpu().numpy())
            act_outputs.append(_.numpy())

    return (np.concatenate(predictions), np.concatenate(act_outputs))

In [13]:
import optuna
def objective(trial):
    #num_layers = params[0], outfeatures = params[1], dim_feedforward = params[2]
    params1 = [trial.suggest_int("num_layers", low = 2, high = 4, step = 1),
              trial.suggest_int("LSTM_neurons", low = 24, high = 48, step = 12)]

    params2 = [trial.suggest_int("num_hiddenZero", low = 24, high = 48, step = 12),
               trial.suggest_int("num_hiddenOne", low = 36, high = 72, step = 18),
               trial.suggest_int("num_hiddenTwo", low = 48, high = 96, step = 24),
               trial.suggest_int("num_hiddenThree", low = 32, high = 64, step = 16),
               trial.suggest_int("num_hiddenFour", low = 24, high = 48, step = 12)]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
    return Train_and_Evaluate(TrainingLoader, ValidationLoader, device, params1, params2, 260, 35);

import joblib
study_name = 'sqlite:///StackedLSTMver2.db'
study = optuna.create_study(direction = "minimize", sampler = optuna.samplers.TPESampler(), study_name = "StackedLSTMver3", load_if_exists = True, storage = 'sqlite:///StackedLSTMver3.db')
joblib.dump(study, "StackedLSTMver2.pkl")
study.optimize(objective, n_trials = 280)


[I 2024-06-21 14:29:56,821] Using an existing study with name 'StackedLSTMver3' instead of creating a new one.
[W 2024-06-21 14:30:01,025] Trial 378 failed with parameters: {'num_layers': 3, 'LSTM_neurons': 24, 'num_hiddenZero': 24, 'num_hiddenOne': 36, 'num_hiddenTwo': 48, 'num_hiddenThree': 48, 'num_hiddenFour': 24} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/jik19004/miniconda3/envs/CLEAN/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_20205/1829702913.py", line 13, in objective
    return Train_and_Evaluate(TrainingLoader, ValidationLoader, device, params1, params2, 260, 35);
  File "/tmp/ipykernel_20205/2292688360.py", line 35, in Train_and_Evaluate
    batchLoss.backward();
  File "/home/jik19004/miniconda3/envs/CLEAN/lib/python3.10/site-packages/torch/_tensor.py", line 488, in backward
    torch.autograd.backward(
  File "/home/jik19

KeyboardInterrupt: 

In [ ]:
print(study.best_params)
print(study.best_value)

{'num_layers': 3, 'LSTM_neurons': 24, 'num_hiddenZero': 24, 'num_hiddenOne': 36, 'num_hiddenTwo': 48, 'num_hiddenThree': 48, 'num_hiddenFour': 24}
123.8967056274414


In [ ]:
params1 = [study.best_params["num_layers"], study.best_params["LSTM_neurons"]]
params2 = [study.best_params["num_hiddenZero"], study.best_params["num_hiddenOne"], study.best_params["num_hiddenTwo"], study.best_params["num_hiddenThree"], study.best_params["num_hiddenFour"]]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");

Train_and_Evaluate(TrainingLoader, ValidationLoader, device, params1, params2, 260, 35)

passed  0 epoch Training Loss:  3026.4539925318377  Validation Loss:  tensor(1879.3081, device='cuda:0')
passed  1 epoch Training Loss:  564.3100102727999  Validation Loss:  tensor(165.5255, device='cuda:0')
passed  2 epoch Training Loss:  227.62173267891794  Validation Loss:  tensor(183.9295, device='cuda:0')
passed  3 epoch Training Loss:  222.7155249726874  Validation Loss:  tensor(174.1448, device='cuda:0')
passed  4 epoch Training Loss:  220.37071762845076  Validation Loss:  tensor(163.7190, device='cuda:0')
passed  5 epoch Training Loss:  214.79747908751446  Validation Loss:  tensor(168.7150, device='cuda:0')
passed  6 epoch Training Loss:  215.3638854012099  Validation Loss:  tensor(162.5921, device='cuda:0')
passed  7 epoch Training Loss:  213.62908669917002  Validation Loss:  tensor(160.9086, device='cuda:0')
passed  8 epoch Training Loss:  211.6064865393709  Validation Loss:  tensor(164.0545, device='cuda:0')
passed  9 epoch Training Loss:  210.2813605752768  Validation Loss:

tensor(128.1256, device='cuda:0')

In [14]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
model = torch.load("/home/jik19004/FilesToRun/StackedEnsemblePred/StackedLSTMver3BEST")
values = predict(model, TestingLoader, torch.device("cuda"))

error = mean_absolute_percentage_error(values[1], values[0])
error2 = mean_absolute_error(values[1], values[0])
print(error)
print(error2)

0.056055516
170.5203


In [15]:
model = torch.load("/home/jik19004/FilesToRun/StackedEnsemblePred/StackedLSTMver3BEST")
values = predict(model, ValidationLoader, torch.device("cuda"))

error = mean_absolute_percentage_error(values[1], values[0])
print(error)
error = mean_absolute_error(values[1], values[0])
print(error)

0.038251944
128.12563


In [16]:
values = predict(model, TrainingLoader, torch.device("cuda"))

error = mean_absolute_percentage_error(values[1], values[0])
print(error)
error = mean_absolute_error(values[1], values[0])
print(error)


0.0656502
232.51476


In [ ]:
print(study.best_params)

{'num_layers': 4, 'last_hidden_neurons': 42, 'LSTM_neurons': 32, 'act_slope': 0.10339207803590009, 'num_hiddenZero': 36, 'num_hiddenOne': 72, 'num_hiddenTwo': 48, 'num_hiddenThree': 64, 'num_hiddenFour': 48}
